In [1]:
import gymnasium as gym
import numpy as np

class rabbitProblem(gym.Env):
    def __init__(self,config=None):
        self.current_steps = 0
        self.grid_size = 4
        self.action_space = gym.spaces.Discrete(5)
        self.observation_space = gym.spaces.Discrete(2)
        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]
    
    def reset(self, *, seed=None, options=None):
        self._max_episode_steps = 1000
        self.current_steps = 0
        self.rabbit_pos = 2
        return 0,{}
    
    def step(self,action):
        self.current_steps+=1
        possible_pos = []
        if self.rabbit_pos-1>=0:
            possible_pos.append(self.rabbit_pos-1)
        if self.rabbit_pos+1<5:
            possible_pos.append(self.rabbit_pos+1)
        next_pos=possible_pos[0]
        if next_pos==action and len(possible_pos)>1:
            next_pos=possible_pos[1]
        self.rabbit_pos=next_pos
        return 1 if next_pos==action else 0,(1 if next_pos==action else 0)-0.1,next_pos==action or self.current_steps>self._max_episode_steps,False,{}
        


In [2]:
env = rabbitProblem()

In [3]:
env.step(1)

(0, -0.1, False, False, {})

In [4]:
from ray.tune.registry import register_env
register_env("rp", rabbitProblem)

2023-09-16 14:13:21,475	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-16 14:13:21,732	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import ray
from ray import tune,air

stopping_criterion = {"training_iteration": 50}
ray.init(ignore_reinit_error=True, log_to_driver=False)
tuner = tune.Tuner(
    "PPO",
    param_space={
        "env": "rp",
        "framework": "torch",
        "num_gpus": 0,
    },
    run_config=air.RunConfig(stop=stopping_criterion)
)
results = tuner.fit()


2023-09-16 14:13:27,167	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


ImportError: PyTorch was specified as the framework to use (via `config.framework('torch')`)! However, no installation was found. You can install PyTorch via `pip install torch`.